# Chain of judge

In [ ]:
def generate_visual_steps(task_description):
    response = client.chat.completions.create(
        model="phi4",  # Generator-Modell
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Mathematikaufgaben in präzise, nummerierte Schritte zerlegt, "
                "die leicht visuell dargestellt werden können. Gib die Antwort als eine Schritt-für-Schritt-Liste zurück, "
                "beginnend mit '1.', '2.', usw. Jede Nummer sollte einen klar definierten Schritt enthalten."
            )},
            {"role": "user", "content": task_description},
        ]
    )
    return response.choices[0].message.content 

In [ ]:
def evaluate_solution(task_description, generated_solution):
    response = client.chat.completions.create(
        model="phi4",  # Bewertungsmodell
        messages=[
            {"role": "system", "content": (
                "Du bist ein Sprachmodell, das Lösungen für Mathematikaufgaben bewertet. "
                "Bewerte die Lösung nicht nur auf Korrektheit, sondern auch darauf, ob sie die folgenden Kriterien erfüllt:\n"
                "1. Die Lösung muss als nummerierte Schritt-für-Schritt-Anleitung vorliegen.\n"
                "2. Die Schritte müssen klar und präzise formuliert sein.\n"
                "3. Begriffe wie 'Koordinatensystem', 'x-Achse', 'y-Achse', 'Vektor' sollten verwendet werden, "
                "wenn sie für die Aufgabe relevant sind.\n"
                "Antworte mit einer kurzen Bewertung und sage klar, ob die Lösung akzeptiert wird oder nicht."
            )},
            {"role": "user", "content": f"Aufgabe:\n{task_description}\n\nLösung:\n{generated_solution}"},
        ]
    )
    return response.choices[0].message.content

In [ ]:
def interactive_solution(task_description, max_iterations=2):
    iteration = 0
    solution = generate_visual_steps(task_description)
    final_solution = None  
    print(f"Iteration {iteration + 1} - Generierte Lösung:\n{solution}")
    
    while iteration < max_iterations:
        evaluation = evaluate_solution(task_description, solution)
        print(f"Iteration {iteration + 1} - Bewertung der Lösung:\n{evaluation}")
        
        if "akzeptiert" in evaluation.lower() and "nummeriert" in evaluation.lower():
            print("Lösung akzeptiert!")
            final_solution = solution
            break
        else:
            iteration += 1
            if iteration < max_iterations:
                print(f"Neue Iteration {iteration + 1}: Verbesserung der Lösung...")
                solution = generate_visual_steps(task_description)
                print(f"Iteration {iteration + 1} - Neue generierte Lösung:\n{solution}")
            else:
                print("Maximale Anzahl an Iterationen erreicht. Keine weiteren Verbesserungen möglich.")
    
    if final_solution is None:
        final_solution = solution
    return final_solution  

In [ ]:
# Beispielaufgabe
task_description = "Zeichne den Punkt P(4|6) in ein Koordinatensystem ein."

# Ablauf mit maximal 2 Iterationen starten
final_solution = interactive_solution(task_description)